In [5]:
import pandas as pd
import json
import os
import numpy as np
import datetime
from haversine import haversine
import zipcodes
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/original_center_of_rentrak_zip.json","r"))
os.getcwd()

# Note: use 13 DMA zips to merge

'/home/jian/Projects/Big_Lots/Analysis/2018_Q4/Zip_For_13_Mkts'

In [6]:
zip_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,dtype=str,usecols=[0,2])
zip_DMA.columns=['zip_cd',"DMA"]
zip_DMA=zip_DMA.drop_duplicates()
DMA_13_List=['ALBANY-SCHENECTADY-TROY','AUSTIN','BIRMINGHAM (ANN AND TUSC)','CHARLOTTE','DETROIT','GREENSBORO-H.POINT-W.SALEM',
             'HARTFORD & NEW HAVEN','INDIANAPOLIS','JACKSONVILLE','KANSAS CITY','OKLAHOMA CITY','PORTLAND, OR','SAN DIEGO']
zip_DMA_13_only=zip_DMA[zip_DMA['DMA'].isin(DMA_13_List)].drop_duplicates()
zip_DMA_list_df=zip_DMA.groupby(['zip_cd'])['DMA'].apply(list).to_frame().reset_index().rename(columns={"DMA":"DMA_list"})
zip_DMA_list_df.head(2)

,zip_cd,DMA_list
0,00501,[NEW YORK]
1,00544,[NEW YORK]


In [7]:
store_level_zips=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
store_level_zips=store_level_zips[store_level_zips['location_id']!="6990"]
store_level_zips=store_level_zips[['location_id','zip','revenue_flag']].drop_duplicates()
store_level_zips=store_level_zips.rename(columns={"zip":"zip_cd"})

In [8]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190101-135843-638.txt",dtype=str,sep="|")
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)


sales_20190105=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-01-05.csv",dtype=str)
sales_20190105=sales_20190105[sales_20190105['week_end_date']=="2019-01-05"]
sales_20190105['sales']=sales_20190105['sales'].astype(float)
sales_20190105=sales_20190105[sales_20190105['sales']>0]

store_list=store_list[store_list['location_id'].isin(sales_20190105['location_id'])]
store_list['zip_cd']=store_list['zip_cd'].apply(lambda x: x.split("-")[0])

store_list=pd.merge(store_list,zip_DMA_13_only,on="zip_cd",how="left") # DMA for stores
store_list_13=store_list[~pd.isnull(store_list['DMA'])]
# store 5371 duplicated in 2 DMAs: CHARLOTTE, GREENSBORO-H.POINT-W.SALEM

# 1. Stores in 13 DMAs, and get all P/S zips for the stores (S 20 miles from the store)

In [9]:
store_level_zips_13_DMAs=store_level_zips[store_level_zips['location_id'].isin(store_list_13['location_id'])]
store_level_zips_13_DMAs_PS=store_level_zips_13_DMAs[store_level_zips_13_DMAs['revenue_flag'].isin(['P','S'])]
store_level_zips_13_DMAs_PS=pd.merge(store_level_zips_13_DMAs_PS,store_list[['location_id','latitude_meas','longitude_meas']],on="location_id",how="left")
print(store_level_zips_13_DMAs_PS.shape)
store_level_zips_13_DMAs_PS=pd.merge(store_level_zips_13_DMAs_PS,store_list_13[['location_id','DMA']],on="location_id",how="left").rename(columns={"DMA":"store_DMA"})
print(store_level_zips_13_DMAs_PS.shape)
store_level_zips_13_DMAs_PS=pd.merge(store_level_zips_13_DMAs_PS,zip_DMA_list_df,on="zip_cd",how="left").rename(columns={"DMA_list":"DMA_for_zip"})

(2800, 5)
(2800, 6)


In [10]:
len(store_level_zips_13_DMAs_PS['zip_cd'].unique())

1717

In [11]:
store_level_zips_13_DMAs_PS['dist_miles']=np.nan
for i in range(len(store_level_zips_13_DMAs_PS)):
    store_loc=(store_level_zips_13_DMAs_PS['latitude_meas'][i],store_level_zips_13_DMAs_PS['longitude_meas'][i])
    zip_code=store_level_zips_13_DMAs_PS['zip_cd'][i]
    if zip_code in zip_centers.keys():
        dist=haversine(store_loc,zip_centers[zip_code],miles=True)
    else:
        try:
            dist=haversine(store_loc,(float(zipcodes.matching(zip_code)['lat']),float(zipcodes.matching(zip_code)['long'])),miles=True)
        except:
            dist=np.nan
    store_level_zips_13_DMAs_PS['dist_miles'][i]=dist
store_level_zips_13_DMAs_PS=store_level_zips_13_DMAs_PS[(store_level_zips_13_DMAs_PS['revenue_flag']=="P") | (store_level_zips_13_DMAs_PS['dist_miles']<=20)]
store_level_zips_13_DMAs_PS['Store_in_13_DMA']=True

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


# 2. Stores in 13 DMAs, but new and get all zips in 10 miles

In [12]:
stores_new=[x for x in store_list_13['location_id'].unique().tolist() if x not in store_level_zips_13_DMAs_PS['location_id'].unique().tolist()]
print(stores_new)

new_store_list=store_list[store_list['location_id'].isin(stores_new)][['location_id','latitude_meas','longitude_meas','DMA']].rename(columns={"DMA":"store_DMA"})
new_store_list['revenue_flag']="zip_in_10_miles"
new_store_list=new_store_list.reset_index()
del new_store_list['index']
new_store_list.head(2)

['4671', '4676', '5363', '5382', '5386', '5371', '5374', '5379']


,location_id,latitude_meas,longitude_meas,store_DMA,revenue_flag
0,4671,41.285692,-72.880966,HARTFORD & NEW HAVEN,zip_in_10_miles
1,4676,33.192350,-117.360965,SAN DIEGO,zip_in_10_miles


In [13]:
zips_for_new_stores=pd.DataFrame()
i_counter=0
for i in range(len(new_store_list)):
    location_id=new_store_list['location_id'][i]
    store_center=(new_store_list['latitude_meas'][i],new_store_list['longitude_meas'][i])
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":location_id,"zip_cd":zip_cd,"dist_miles":dist},index=[i_counter])
            i_counter+=1
            zips_for_new_stores=zips_for_new_stores.append(df)
            
    
            
    

In [14]:
new_store_list=pd.merge(new_store_list,zips_for_new_stores,on="location_id",how="left")
new_store_list['Store_in_13_DMA']=True
new_store_list=pd.merge(new_store_list,zip_DMA_list_df,on="zip_cd",how="left").rename(columns={"DMA_list":"DMA_for_zip"})

# 3. zips for other stores out of 13 DMAs, but labeled as P/S (S 20 miles from the store)

In [15]:
store_level_zips=pd.merge(store_level_zips,zip_DMA,on="zip_cd",how="left")
store_level_zips=store_level_zips[~store_level_zips['zip_cd'].isin(new_store_list['zip_cd'].unique().tolist()+store_level_zips_13_DMAs_PS['zip_cd'].unique().tolist())]
store_level_zips=store_level_zips[store_level_zips['DMA'].isin(DMA_13_List)]
store_level_zips=store_level_zips[store_level_zips['revenue_flag'].isin(["P","S"])]

zips_in_store_out_df=store_level_zips[['location_id','zip_cd','revenue_flag']].drop_duplicates()
zips_in_store_out_df['Store_in_13_DMA']=False
zips_in_store_out_df=pd.merge(zips_in_store_out_df,store_list[['location_id','latitude_meas','longitude_meas']],on="location_id",how="left")
zips_in_store_out_df=pd.merge(zips_in_store_out_df,zip_DMA_list_df,on="zip_cd",how="left").rename(columns={"DMA_list":"DMA_for_zip"})
zips_in_store_out_df['store_DMA']="Not_in_the_13_DMAs"
zips_in_store_out_df=zips_in_store_out_df.reset_index()
del zips_in_store_out_df['index']

In [16]:
zips_in_store_out_df.head(2)

,location_id,zip_cd,revenue_flag,Store_in_13_DMA,latitude_meas,longitude_meas,DMA_for_zip,store_DMA
0,1006,29520,S,False,33.678309,-78.90332,[CHARLOTTE],Not_in_the_13_DMAs
1,1006,29706,S,False,33.678309,-78.90332,[CHARLOTTE],Not_in_the_13_DMAs


In [18]:
zips_in_store_out_df.shape

(243, 8)

In [19]:
zips_in_store_out_df['dist_miles']=np.nan
for i in range(len(zips_in_store_out_df)):
    location_id=zips_in_store_out_df['location_id'][i]
    store_center=(zips_in_store_out_df['latitude_meas'][i],zips_in_store_out_df['longitude_meas'][i])
    zip_code=zips_in_store_out_df['zip_cd'][i]
    if zip_code in zip_centers.keys():
        dist=haversine(store_loc,zip_centers[zip_code],miles=True)
    else:
        try:
            dist=haversine(store_loc,(float(zipcodes.matching(zip_code)['lat']),float(zipcodes.matching(zip_code)['long'])),miles=True)
        except:
            dist=np.nan
    zips_in_store_out_df['dist_miles'][i]=dist
zips_in_store_out_df=zips_in_store_out_df[(zips_in_store_out_df['revenue_flag']=="P") | (zips_in_store_out_df['dist_miles']<=20)]      

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [20]:
output_1_store_level=store_level_zips_13_DMAs_PS.append(new_store_list).append(zips_in_store_out_df)
output_1_store_level=output_1_store_level[['zip_cd','location_id','revenue_flag','latitude_meas','longitude_meas',
                                           'store_DMA','Store_in_13_DMA','dist_miles','DMA_for_zip']]

output_1_store_level=output_1_store_level[output_1_store_level['location_id'].isin(sales_20190105['location_id'].tolist())]

In [21]:
output_2_DMA_level=pd.DataFrame()
df_13_dma_stores=output_1_store_level[output_1_store_level['store_DMA']!="Not_in_the_13_DMAs"]
app_other_dma_stores=output_1_store_level[output_1_store_level['store_DMA']=="Not_in_the_13_DMAs"]

for dma,group in df_13_dma_stores.groupby(['store_DMA']):
    group=group.sort_values('revenue_flag').drop_duplicates('zip_cd')
    df_dma_store=app_other_dma_stores[app_other_dma_stores['DMA_for_zip'].apply(lambda x: dma in x)]
    group=group.append(df_dma_store)
    output_2_DMA_level=output_2_DMA_level.append(group)


In [22]:
output_3_total_DMAs_level=output_1_store_level.sort_values('revenue_flag').drop_duplicates(['zip_cd'])

In [23]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/Zip_For_13_Mkts/BL_zips_for_13_test_Mkts_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_3_total_DMAs_level.to_excel(writer,"overall_zips",index=False)
output_2_DMA_level.to_excel(writer,"DMA_level",index=False)
output_1_store_level.to_excel(writer,"Store_level",index=False)
writer.save()

In [24]:
x=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
x=x[x['location_id']=="132"]
x.head(2)

,net_transaction_amt,transactions,zip,revenue_flag,transaction_flag,location_id
156024,347448.77,448,47909,P,P,132
156025,324048.09,402,47905,P,P,132


In [26]:
x[x['zip']=="47906"]


,net_transaction_amt,transactions,zip,revenue_flag,transaction_flag,location_id
156027,118849.66,382,47906,P,P,132
